# 3. Working with the Command Line

Interacting with the shell environment within Python and creating Python command-line tools are both necessary when doing DevOps work


# Working with the Shell

Get good with `sys`, `os` and `subprocess` packages.

## Talking to the Interpreter with the `sys` Module

* Offers access to vairables and methods closely tied to the Python interpreter
* 2 Dominant ways to interpret bytes during reading:
	1. _little endian_: interpret each subsequent byte with higher significance
	2. _big endian_: assumes the first byte has the highest significance and moves down from there


In [2]:
# See the byte order of current architecture
import sys
print(sys.byteorder)

# See size of the Python objects. Good for dealing with limited memory
print(sys.getsizeof(1))

# Check current os working on
print(sys.platform)

# Check the current python version. Good to control different functions for diff versions
if sys.version_info.major < 3:
    print("You need to update your Python")
elif sys.version_info.minor < 7:
    print("You are not running 3.7 version of Python")
else:
    print("All is good")

little
28
win32
All is good


## Dealing with Operating Systems with `os`

`os` module contains various attributes and functions that help deal with operating systems. Most common usage is to get settings from the environment variables (e.g setting logging levels, or getting secrets like API keys)

In [ ]:
import os

# Get current working directory
os.getcwd()

# Change current working directory
os.chdir("/tmp")

# Holds the environment variables that were set when os was loaded
os.environ.get("LOGLEVEL")

# The setting and environment variable. Setting exists for subprocesses spawned from code
os.environ["LOGLEVEL"] = "DEBUG"

# Get login of user in the terminal
os.getlogin()

## Spawn Processes with `subprocess` module

For running applications outside the Python script (e.g Bash scripts, Shell commands, etc). `subprocess` module spawns a new process to run commands within it and collect the output from within Python

Will usually use `subprocess.run` which returns `CompletedProcess` instance once completed.

Some parameters:
* `capture_output`: save the output of the command in a variable
* `universal_newlines`: create newlines for the output given the system
* `check`: automatically checks for an error in the command and returns a CalledProcessError

In [6]:
import subprocess
# Checking current directory
cp = subprocess.run(['ls', '-l'], capture_output=True, universal_newlines=True)
print(cp.stdout)

# Checking current directory for nonexistant file
cp = subprocess.run(['ls', '/doesnotexist'], capture_output=True, universal_newlines=True, check=True)

total 0
-rwxrwx---+ 1 Martin Ho None 0 May 30 21:16 ch3 notes.ipynb



CalledProcessError: Command '['ls', '/doesnotexist']' returned non-zero exit status 2.

# Creating Command-Line Tools

Invoke a Python script on the command line is to invoke using Python. Any statement at the top level (not nested in code blocks) run whenever the script is invoked or imported. Functions at the top level will run when the code is loaded. __Functions will run when invoked on the command line and when imported__.

However, code will run when you import modules, no control over when the content is invoked. Convention for modules running on the command line is to end with a blcok testing for the `main` name, making sure it is only invoked when on the command line and not during import

In [ ]:
def say_it():
    print("Greetings human:")

if __name__ == "__main__":
    say_it()

Introduce command-line arguments to configure how the function runs. They represent the user interface for people using tools.

## Using `sys.argv`

Introduce a slit of arguments passed to a Python script at runtime. First argument is the anem of the script and the rest are any remaining command-line arguments, represented as strings

In [8]:
!python args.py --a-flag some-value 13

The first argument is: 'args.py'
The second argument is: '--a-flag'
The third argument is: 'some-value'
The fourth argument is: '13'


In [ ]:
# Example of argument parser
import sys

def say_it(greeting, target):
    message = f"{greeting} {target}"
    print(message)

if __name__ == "__main__":
    greeting = "Hello"
    target = "Joe"
    
    if "--help" in sys.argv:
        help_message = f"Usage: {sys.argv[0]} --name <NAME> -- greeting <GREETING>"
        print(help_message)
        sys.exit()
    
    if '--name' in sys.argv:
        # get the content after the name flag
        name_index = sys.argv.index("--name") + 1
        if name_index < len(sys.argv):
            name = sys.argv[name_index]
    
    if '--greeting' in sys.argv:
        # get the content after the name flag
        greeting_index = sys.argv.index("--greeting") + 1
        if greeting_index < len(sys.argv):
            greeting = sys.argv[greeting_index]
            
    say_it(name, greeting)

In [9]:
!python simple_greeting.py --name Martin --greeting Howdy!

Martin Howdy!


__Limitations to above code__

* No indication if user misspells
* No indication if use commands that are not supported 
* Miscapitalized flags are ignored

Try not to use `argv` parsing approach for production code. Use other packages that are designed for the creation of command-line tools. Help to design the user interface for the module.

1. `argparse`
2. `click`
3. `python-fire`

## Using `argparse`

Design command-line user interface in detail, defining comamnds and flags along with help messages. Creates parser objects that you attach commands and flags. The parser then parses the arguments and you use the results to call the code

If the name begins with a dash, it is treated as an optional flag, else it is a position-dependent command

In [ ]:
import argparse

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Echo your input")
    parser.add_argument(
		"message",
		help='Message to echo'
	)
    parser.add_argument(
		"--twice",
		'-t',
		help='Do it twice',
		action='store_true' # stores the argument as a boolean value
	)
    
    args = parser.parse_args()
    print(args.message)
    if args.twice:
        print(args.message)

In [10]:
!python simple_parse.py "Hello nice to meet you" -t

Hello nice to meet you
Hello nice to meet you


In [11]:
!python simple_parse.py --help

usage: simple_parse.py [-h] [--twice] message

Echo your input

positional arguments:
  message      Message to echo

optional arguments:
  -h, --help   show this help message and exit
  --twice, -t  Do it twice


Create subcommands by creating subparsers under the main parser. Create a hierarchy of commands using subparsers. Below is a maritime application that has subcommands for ships and sailors with 2 subparsers and a main parser

In [4]:
import argparse

def sail():
	ship_name = "Your ship"
	print(f"{ship_name} is setting sail")

def list_ships():
	ships = ['John B','Yankee Clippers', 'Pequod']
	print(f"Ships: {','.join(ships)}")

def greet(greeting, name):
	message = f"{greeting} {name}"
	print(message)

if __name__ == "__main__":
	parser = argparse.ArgumentParser(description="Maritime control")
	parser.add_argument(
		'--twice',
		'-t',
		help='Do it twice',
		action='store_true'
	)

	# Create the subparsers namespace
	subparsers = parser.add_subparsers(dest='func')

	ship_parser = subparsers.add_parser('ships', help='Ship related commands')
	ship_parser.add_argument('command', choices=['list', 'sail'])

	sailor_parser = subparsers.add_parser('sailors', help='Talk to a sailor')
	sailor_parser.add_argument('name', help='Sailors name')
	sailor_parser.add_argument('--greeting', '-g', help='Greeting', default='Ahoy there')
	
	args = parser.parse_args()

	if args.func == "sailors":
		greet(args.greeting, args.name)
	elif args.command == 'list':
		list_ships()
	else:
		sail()

usage: ipykernel_launcher.py [-h] [--twice] {ships,sailors} ...
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"f5acc992-ac05-45f4-a08c-f62df66363aa" --shell=9002 --transport="tcp" --iopub=9004 --f=c:\Users\P0508911\AppData\Roaming\jupyter\runtime\kernel-186685i0yVCXe23fd.json


SystemExit: 2

c:\Users\P0508911\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3405: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [23]:
!python argparse_example.py ships list

Ships: John B,Yankee Clippers,Pequod


In [26]:
!python argparse_example.py ships sail

Your ship is setting sail


In [46]:
!python argparse_example.py sailors Martin --g Wassup 

Wassup Martin


## Using `click`

Uses _decorators_ to bind the command-line interface directly to functions, weaving decisions directly into the rest of the code. Tie flags and options directly into the parameters of the functions that are exposed.

Reduces the amount of code needed to configure the parser interface and focus on the business logic of the parser

In [43]:
# simple example of a decorator

# w/o decorators, have to initialise the decorators as a function with the wrapped function as an argument
def decorator(some_function):
	def wrapper():
		print("Print before the wrapped function")
		some_function()
		print("Print after the wrapped function")
	return wrapper
def wrapped_func():
	print("Method 1 w/o decorators")
f = decorator(wrapped_func)
f()

# with decorators, you can call the wrapped function with the decorator on top
@decorator
def wrapped_func_deco():
	print("Method 2 w/ decorator")

wrapped_func_deco()

Print before the wrapped function
Method 1 w/o decorators
Print after the wrapped function
Print before the wrapped function
Method 2 w/ decorator
Print after the wrapped function


In [ ]:
# Simple click example
import click

@click.command() # function should be exposed to command-line access
@click.option('--greeting', default='Hiya', help='How do you want to greet') # adds aragument to the command-line
@click.option('--name', default='Tammy', help='Who do you want to greet?')
def greet(greeting, name):
	print(f"{greeting} {name}")

if __name__ == "__main__":
	greet()

In [44]:
!python click_example.py --help

Usage: click_example.py [OPTIONS]

Options:
  --greeting TEXT  How do you want to greet
  --name TEXT      Who do you want to greet?
  --help           Show this message and exit.


In [45]:
!python click_example.py --greeting "Hi, nice to meet you" --name "Jane"

Hi, nice to meet you Jane


In [ ]:
# More complicated example
import click

# create a top-level group which other groups and commands will reside
@click.group() 
def cli(): # function act as top-level group, transforms function into group
	pass

@click.group(help='Ship related commands')
def ships():
	pass

cli.add_command(ships)

@ships.command(help="Sail a ship")
def sail():
	ship_name = 'Your ship'
	print(f"{ship_name} is setting sail")
@ships.command(help='List all of the ships')
def list_ships():
	ships = ['John B', 'Yankee Clipper', 'Prequod']
	print(f"Ships: {','.join(ships)}")

@cli.command(help='Talk to a sailor')
@click.option('--greeting', default='Ahoy there', help='Greeting from a sailor')
@click.argument('name')
def sailors(greeting, name):
	print(f"{greeting} {name}")

if __name__ == "__main__":
	cli()

In [49]:
!python another_click_example.py --help

Usage: another_click_example.py [OPTIONS] COMMAND [ARGS]...

Options:
  --help  Show this message and exit.

Commands:
  sailors  Talk to a sailor
  ships    Ship related commands


In [51]:
!python another_click_example.py ships list-ships

Ships: John B,Yankee Clipper,Prequod


In [52]:
!python another_click_example.py ships sail

Your ship is setting sail


In [55]:
!python another_click_example.py sailors "Martin" --greeting "First Mate,"

First Mate, Martin


In [56]:
# pg 146